In [126]:
import os 
import numpy as np
import glob
import pandas as pd

In [127]:
#set path 
pd_path = "/home/olle/PycharmProjects/clinical_feature_segmentation/oct_feature_log.csv"

#read csv
oct_pd = pd.read_csv(pd_path, index_col=0)

#create patient id columns
oct_pd["pat_id"] = oct_pd.img.str.split("/",expand=True)[5].str.split("_",expand=True)[0]

In [ ]:
### feature 1 - epiratinal membrane -50 epiretinal membrance OCT's

In [128]:
epi_examples = oct_pd[oct_pd["1"]==1].\
groupby('pat_id',as_index=False).apply(lambda df: df.sample(1)).iloc[0:50]#[["img"]]

In [ ]:
### feature 3 - intra retinal fluid - 100  OCT's

In [129]:
intr_fluid_ = oct_pd[oct_pd["3"]==1].\
groupby('pat_id',as_index=False).apply(lambda df: df.sample(1)).iloc[0:100]#[["img"]]

In [ ]:
### feature 4 - sub retinal fluid

In [138]:
subr_fluid_ = oct_pd[oct_pd["4"]==1].\
groupby('pat_id',as_index=False).apply(lambda df: df.sample(1)).iloc[0:400]#[["img"]]

In [ ]:
### feature 5 - subretinal hyper reflective material

In [143]:
subhrm_ = oct_pd[oct_pd["5"]==1].\
groupby('pat_id',as_index=False).apply(lambda df: df.sample(1)).iloc[0:400]#[["img"]]

In [ ]:
### feature 6 - RPE

In [153]:
rpe_ = oct_pd[oct_pd["6"]==1].\
groupby('pat_id',as_index=False).apply(lambda df: df.sample(1)).iloc[0:400]#[["img"]]

rpe_.shape

(400, 13)

In [ ]:
### feature 7 - fibrovascular PDE

In [161]:
fvPDE_ = oct_pd[oct_pd["7"]==1].\
groupby('pat_id',as_index=False).apply(lambda df: df.sample(1)).iloc[0:100]#[["img"]]

fvPDE_.shape

(82, 13)

In [ ]:
### feature 8 - geogrphic atrophy

In [174]:
geo_a_ = oct_pd[oct_pd["8"]==1].\
groupby('pat_id',as_index=False).apply(lambda df: df.sample(1)).iloc[0:500]#[["img"]]

geo_a_.shape

(135, 13)

In [ ]:
### feature 9 - posterior hylaoid membrane

In [204]:
d_ped_ = oct_pd[oct_pd["9"]==1].\
groupby('pat_id',as_index=False).apply(lambda df: df.sample(1)).iloc[0:500]#[["img"]]

d_ped_.shape

(50, 13)

In [ ]:
### join tables excluding re-occuring patients

In [206]:
main_ = epi_examples[["img","pat_id"]]
main_.reset_index(drop=True, inplace=True)

#feature 3
exclusion_irf = main_.merge(intr_fluid_,
         left_on="pat_id",right_on="pat_id",
         how="outer", indicator=True)[["img_y","pat_id","_merge"]]

main_irf = exclusion_irf[exclusion_irf._merge=="right_only"][["img_y",
                                                              "pat_id"]].rename(columns={"img_y":"img"})

main_ = pd.concat([main_,main_irf])

#feature 4
exclusion_srf = main_.merge(subr_fluid_,
         left_on="pat_id",right_on="pat_id",
         how="outer", indicator=True)[["img_y","pat_id","_merge"]]

main_srf = exclusion_srf[exclusion_srf._merge=="right_only"][["img_y",
                                                              "pat_id"]].rename(columns={"img_y":"img"})

main_ = pd.concat([main_,main_srf])

#feature 5
exclusion_srhrm = main_.merge(subhrm_,
         left_on="pat_id",right_on="pat_id",
         how="outer", indicator=True)[["img_y","pat_id","_merge"]]

main_srhrm = exclusion_srhrm[exclusion_srhrm._merge=="right_only"][["img_y",
                                                              "pat_id"]].rename(columns={"img_y":"img"})

main_ = pd.concat([main_,main_srhrm])

#feature 6
exclusion_rpe = main_.merge(rpe_,
         left_on="pat_id",right_on="pat_id",
         how="outer", indicator=True)[["img_y","pat_id","_merge"]]

main_rpe = exclusion_rpe[exclusion_rpe._merge=="right_only"][["img_y",
                                                              "pat_id"]].rename(columns={"img_y":"img"})

main_ = pd.concat([main_,main_rpe])

#feature 7 -  all patients allready present
exclusion_fvPDE = main_.merge(fvPDE_,
         left_on="pat_id",right_on="pat_id",
         how="outer", indicator=True)[["img_y","pat_id","_merge"]]

#here setting "both" as otherwise all records get excluded
main_fvPDE = exclusion_fvPDE[exclusion_fvPDE._merge=="both"][["img_y",
                                                              "pat_id"]].rename(columns={"img_y":"img"})

main_ = pd.concat([main_,main_fvPDE])

#feature 8 -  all patients allready present
exclusion_geo_a = main_.merge(geo_a_,
         left_on="pat_id",right_on="pat_id",
         how="outer", indicator=True)[["img_y","pat_id","_merge"]]

#here setting "both" as otherwise all records get excluded
main_geo_a = exclusion_geo_a[exclusion_geo_a._merge=="both"][["img_y",
                                                              "pat_id"]].rename(columns={"img_y":"img"})[0:100]


#feature 9 -  all patients allready present
exclusion_d_ped_ = main_.merge(d_ped_,
         left_on="pat_id",right_on="pat_id",
         how="outer", indicator=True)[["img_y","pat_id","_merge"]]

#here setting "both" as otherwise all records get excluded
main_d_ped_ = exclusion_d_ped_[exclusion_d_ped_._merge=="both"][["img_y",
                                                              "pat_id"]].rename(columns={"img_y":"img"})[0:100]



main_ = pd.concat([main_,main_d_ped_])


### second iteration of OCT's for annotation

In [213]:
main_unique = main_.drop_duplicates()

print("number of records:",main_unique.shape[0])

number of records: 529


In [215]:
main_unique.to_csv("oct_second_iteration.csv")